In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests
from datetime import datetime as dt
import plotly.subplots as sp
from pandas.tseries.offsets import MonthEnd

In [2]:
eia_html = requests.get('https://www.eia.gov/electricity/data/eia923/')
soup = BeautifulSoup(eia_html.content,'html.parser')
release_dates= soup.find_all('div',class_='release-dates')


In [3]:
next_update_time = release_dates[0].find_all('span',class_='date')[-1].text

In [4]:
monthly_report_links = {}

In [5]:
this_year =dt.today().year

In [6]:
dates_to_observe = range(this_year-5,this_year+1)

In [7]:
for x in soup.find_all('a', href=True):
    try:
        if int(x.attrs['title']) in dates_to_observe:
            monthly_report_links[x.attrs['title']] = x.attrs['href']
    except:
        pass

In [8]:
link_df = pd.DataFrame.from_dict(monthly_report_links,orient='index').sort_index()

In [9]:
from zipfile import ZipFile
from io import BytesIO
from urllib.request import urlopen

from RGGI_plant_analysis import RGGI_capacity

In [10]:
mega_gen_fuel_df = pd.DataFrame()
for year in link_df.index:
    resp = urlopen('https://www.eia.gov/electricity/data/eia923/'+link_df.loc[year][0])
    myzip = ZipFile(BytesIO(resp.read()))
    for  x in myzip.filelist:
        if 'EIA923_Schedules_2_3_4_5_M' in x.filename:
            print(x.filename)
            gen_file = pd.ExcelFile(myzip.open(x))
            gen_fuel_df = pd.read_excel(gen_file,sheet_name='Page 1 Generation and Fuel Data',header=5)
            gen_fuel_df = gen_fuel_df.dropna(subset=['Plant Name'])

            mega_gen_fuel_df = pd.concat([mega_gen_fuel_df,gen_fuel_df],axis=0)

EIA923_Schedules_2_3_4_5_M_12_2019_Final_Revision.xlsx
Page 1 Energy Storage
Page 1 Generation and Fuel Data
Page 1 Puerto Rico
Page 2 Coal Stocks Data
Page 2 Oil Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 6 Plant Frame Puerto Rico
EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx
Page 1 Generation and Fuel Data
Page 1 Energy Storage
Page 1 Puerto Rico
Page 2 Coal Stocks Data
Page 2 Oil Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6 Plant Frame
Page 6 Plant Frame Puerto Rico
EIA923_Schedules_2_3_4_5_M_12_2021_Final_Revision.xlsx
Page 1 Energy Storage
Page 1 Generation and Fuel Data
Page 1 Puerto Rico
Page 2 Coal Stocks Data
Page 2 Oil Stocks Data
Page 2 Petcoke Stocks Data
Page 2 Stocks Data
Page 3 Boiler Fuel Data
Page 4 Generator Data
Page 5 Fuel Receipts and Costs
Page 6

In [11]:
### Capacity
def save_historical_capacity_per_plant():
    time_series_historical_capacity = pd.DataFrame()
    for x in range(1,12*6):
        next_update_time,recent_report,report_month,report_year  =RGGI_capacity().scrape_recent_EIA_860m(lagged_report=x)
        capacity = RGGI_capacity().analyse_all_capacity(recent_report,report_month,report_year)
        time_series_historical_capacity = pd.concat([time_series_historical_capacity,capacity],axis=0)

    time_series_historical_capacity.to_pickle('full_capacity_series.pkl')

    return time_series_historical_capacity

In [12]:
#time_series_historical_capacity = save_historical_capacity_per_plant()

In [13]:
time_series_historical_capacity = pd.read_pickle('full_capacity_series.pkl')

In [14]:
time_series_historical_capacity

Nameplate Capacity (MW)
Plant ID Energy Source Code Prime Mover Code Plant State report_month report_year                         
1.0      DFO                IC               AK          9            2024                             2.7
         WND                WT               AK          9            2024                             1.0
2.0      WAT                HY               AL          9            2024                            53.9
3.0      BIT                ST               AL          9            2024                          1192.5
         NG                 CA               AL          9            2024                           700.4
...                                                                                                    ...
62264.0  SUN                PV               MA          11           2018                             1.2
62265.0  SUN                PV               MA          11           2018                             1.9
62266.0  SUN                PV               MA          11           2018                             1.1
62270.0  DFO                IC               NY          11           2018                             8.0
62271.0  DFO                IC               NY          11           2018                             4.5

[879099 rows x 1 columns]

In [15]:
gen_cols = list(mega_gen_fuel_df.columns[['Netgen' in x for x in mega_gen_fuel_df.columns]])

In [16]:
mega_gen_fuel_df.query('`Plant Id`==1353')

,Plant Id,Combined Heat And\nPower Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,...,Netgen\nNovember,Netgen\nDecember,Total Fuel Consumption\nQuantity,Electric Fuel Consumption\nQuantity,Total Fuel Consumption\nMMBtu,Elec Fuel Consumption\nMMBtu,Net Generation\n(Megawatthours),YEAR,MER\nFuel Type Code,Reserved.1
1286,1353,N,.,Big Sandy,Kentucky Power Co,22053,KY,ESC,RFC,NaN,...,42322,101435,8664644,8664644,10603604,10603604,1062894.0,2019,NaN,NaN
1234,1353,N,.,Big Sandy,Kentucky Power Co,22053,KY,ESC,RFC,NaN,...,89953,71690,7349536,7349536,9042027,9042027,912638.0,2020,NaN,NaN
1219,1353,N,.,Big Sandy,Kentucky Power Co,22053,KY,ESC,RFC,NaN,...,0,33581,4662496,4662496,5668700,5668700,550541.0,2021,NaN,NaN
1209,1353,N,.,Big Sandy,Kentucky Power Co,22053,KY,ESC,RFC,NaN,...,0,0,4362565,4362565,5256316,5256316,509951.0,2022,NG,NaN
1200,1353,N,.,Big Sandy,Kentucky Power Co,22053,KY,ESC,RFC,NaN,...,85554,74131,9881288,9881288,11922022,11922022,1172392.0,2023,NG,NaN


In [17]:
#gen_cols.append('YEAR')

In [18]:
gen_cols

['Netgen\nJanuary',
 'Netgen\nFebruary',
 'Netgen\nMarch',
 'Netgen\nApril',
 'Netgen\nMay',
 'Netgen\nJune',
 'Netgen\nJuly',
 'Netgen\nAugust',
 'Netgen\nSeptember',
 'Netgen\nOctober',
 'Netgen\nNovember',
 'Netgen\nDecember']

In [19]:
RGGI_states = ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'RI', 'VT'] 

In [20]:
mega_gen_fuel_df['RGGI_state'] = np.where(mega_gen_fuel_df['Plant State'].isin(RGGI_states),1,0)

In [21]:
mega_gen_fuel_df['Reported\nPrime Mover'].unique()

array(['IC', 'WT', 'HY', 'CA', 'CT', 'ST', 'GT', 'PS', 'CS', 'PV', 'BA',
       'CE', 'BT', 'OT', 'FC', 'CP', 'FW', 'WS', 'WY'], dtype=object)

In [22]:
mega_gen_fuel_df['Reported\nFuel Type Code'].unique()

array(['DFO', 'WND', 'WAT', 'NG', 'BIT', 'SUB', 'NUC', 'LIG', 'PG', 'RC',
       'AB', 'WDS', 'RFO', 'LFG', 'PC', 'SUN', 'OBG', 'GEO', 'MWH', 'OG',
       'WO', 'JF', 'KER', 'OTH', 'WC', 'SGC', 'OBS', 'TDF', 'BFG', 'MSB',
       'MSN', 'SC', 'BLQ', 'WH', 'OBL', 'SLW', 'PUR', 'WDL', 'SGP', 'ANT',
       'H2'], dtype=object)

In [23]:
mega_gen_fuel_df['AER\nFuel Type Code'].unique()

array(['DFO', 'WND', 'HYC', 'NG', 'COL', 'NUC', 'WOO', 'ORW', 'WWW',
       'HPS', 'RFO', 'MLG', 'PC', 'SUN', 'GEO', 'OTH', 'OOG', 'WOC', nan],
      dtype=object)

In [24]:
rggi_or_pjm = mega_gen_fuel_df[(mega_gen_fuel_df['RGGI_state']==1)| (mega_gen_fuel_df['Balancing\nAuthority Code']=='PJM')]

In [25]:
rggi_or_pjm['Plant State'].unique()

array(['KY', 'NY', 'IL', 'CT', 'MA', 'VT', 'DE', 'MD', 'ME', 'IN', 'MI',
       'NH', 'NJ', 'NC', 'OH', 'PA', 'RI', 'VA', 'WV', 'TN', 'DC', 'MN'],
      dtype=object)

In [26]:
gen_cols

['Netgen\nJanuary',
 'Netgen\nFebruary',
 'Netgen\nMarch',
 'Netgen\nApril',
 'Netgen\nMay',
 'Netgen\nJune',
 'Netgen\nJuly',
 'Netgen\nAugust',
 'Netgen\nSeptember',
 'Netgen\nOctober',
 'Netgen\nNovember',
 'Netgen\nDecember']

In [27]:
rggi_or_pjm.columns

Index(['Plant Id', 'Combined Heat And\nPower Plant', 'Nuclear Unit Id',
       'Plant Name', 'Operator Name', 'Operator Id', 'Plant State',
       'Census Region', 'NERC Region', 'Reserved', 'NAICS Code',
       'EIA Sector Number', 'Sector Name', 'Reported\nPrime Mover',
       'Reported\nFuel Type Code', 'AER\nFuel Type Code',
       'Balancing\nAuthority Code', 'Respondent\nFrequency',
       'Physical\nUnit Label', 'Quantity\nJanuary', 'Quantity\nFebruary',
       'Quantity\nMarch', 'Quantity\nApril', 'Quantity\nMay', 'Quantity\nJune',
       'Quantity\nJuly', 'Quantity\nAugust', 'Quantity\nSeptember',
       'Quantity\nOctober', 'Quantity\nNovember', 'Quantity\nDecember',
       'Elec_Quantity\nJanuary', 'Elec_Quantity\nFebruary',
       'Elec_Quantity\nMarch', 'Elec_Quantity\nApril', 'Elec_Quantity\nMay',
       'Elec_Quantity\nJune', 'Elec_Quantity\nJuly', 'Elec_Quantity\nAugust',
       'Elec_Quantity\nSeptember', 'Elec_Quantity\nOctober',
       'Elec_Quantity\nNovember', 'Ele

In [28]:
fuel_cons_columns = ['Quantity\nJanuary', 'Quantity\nFebruary',
       'Quantity\nMarch', 'Quantity\nApril', 'Quantity\nMay', 'Quantity\nJune',
       'Quantity\nJuly', 'Quantity\nAugust', 'Quantity\nSeptember',
       'Quantity\nOctober', 'Quantity\nNovember', 'Quantity\nDecember']

mmbtu_cols = ['Tot_MMBtu\nJanuary', 'Tot_MMBtu\nFebruary', 'Tot_MMBtu\nMarch',
       'Tot_MMBtu\nApril', 'Tot_MMBtu\nMay', 'Tot_MMBtu\nJune',
       'Tot_MMBtu\nJuly', 'Tot_MMBtu\nAugust', 'Tot_MMBtu\nSeptember',
       'Tot_MMBtu\nOctober', 'Tot_MMBtu\nNovember', 'Tot_MMBtu\nDecember']

In [29]:
rggi_or_pjm[gen_cols] = rggi_or_pjm[gen_cols].replace('.',0)
rggi_or_pjm[fuel_cons_columns] = rggi_or_pjm[fuel_cons_columns].replace('.',0)
rggi_or_pjm[mmbtu_cols] = rggi_or_pjm[mmbtu_cols].replace('.',0)

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_19200\3169541333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rggi_or_pjm[gen_cols] = rggi_or_pjm[gen_cols].replace('.',0)
C:\Users\Alex Child\AppData\Local\Temp\ipykernel_19200\3169541333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rggi_or_pjm[fuel_cons_columns] = rggi_or_pjm[fuel_cons_columns].replace('.',0)
C:\Users\Alex Child\AppData\Local\Temp\ipykernel_19200\3169541333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [30]:
mmbtu_df = rggi_or_pjm.groupby(['Plant Id','Plant State','Reported\nPrime Mover','Reported\nFuel Type Code','YEAR'])[mmbtu_cols].sum().stack().to_frame('Fuel_consumed_Mmbtu')
mmbtu_df['month'] = [x[-1] for x in mmbtu_df.reset_index()['level_5'].str.split('\n')]
mmbtu_df=mmbtu_df.reset_index()
mmbtu_df['Date'] = pd.to_datetime('01/'+mmbtu_df['month']+'/'+mmbtu_df['YEAR'].astype(str))+MonthEnd(0)
mmbtu_df['month'] = mmbtu_df['Date'].dt.month

In [31]:
"""
fuel_cons_df = rggi_or_pjm.groupby(['Plant Id','Plant State','Reported\nPrime Mover','Reported\nFuel Type Code','Physical\nUnit Label','YEAR'])[fuel_cons_columns].sum().stack().to_frame('Fuel_consumed_q')
fuel_cons_df['month'] = [x[-1] for x in fuel_cons_df.reset_index()['level_6'].str.split('\n')]
fuel_cons_df=fuel_cons_df.reset_index()
fuel_cons_df['Date'] = pd.to_datetime('01/'+fuel_cons_df['month']+'/'+fuel_cons_df['YEAR'].astype(str))+MonthEnd(0)
fuel_cons_df['month'] = fuel_cons_df['Date'].dt.month"""

"\nfuel_cons_df = rggi_or_pjm.groupby(['Plant Id','Plant State','Reported\nPrime Mover','Reported\nFuel Type Code','Physical\nUnit Label','YEAR'])[fuel_cons_columns].sum().stack().to_frame('Fuel_consumed_q')\nfuel_cons_df['month'] = [x[-1] for x in fuel_cons_df.reset_index()['level_6'].str.split('\n')]\nfuel_cons_df=fuel_cons_df.reset_index()\nfuel_cons_df['Date'] = pd.to_datetime('01/'+fuel_cons_df['month']+'/'+fuel_cons_df['YEAR'].astype(str))+MonthEnd(0)\nfuel_cons_df['month'] = fuel_cons_df['Date'].dt.month"

In [32]:
mmbtu_df = mmbtu_df.groupby(['Plant Id','Plant State','Reported\nPrime Mover','Date','Reported\nFuel Type Code'])['Fuel_consumed_Mmbtu'].sum().unstack('Reported\nFuel Type Code').fillna(0)#.add_suffix('_mmbtu')

In [33]:
mmbtu_df.loc[55375,:].loc[:,:,'2024-09-30'][['NG','DFO']]

Reported
Fuel Type Code                   NG    DFO
Plant State Reported\nPrime Mover                  
NY          CA                      160903.0    0.0
            CT                     2424223.0  203.0

In [34]:
55375

55375

In [35]:
from EIA_emissions_factors import download_EF_from_EIA

In [36]:
EF_df = download_EF_from_EIA(fuels_list=mmbtu_df.columns)

In [37]:
plant_emissions = mmbtu_df.dot(EF_df.astype(float))

In [38]:
plant_emissions['Emissions_stCO2']=(plant_emissions/1000)* 0.9071847

In [39]:
plant_emissions.loc[55375,:].loc[:,:,'2024-09-30']

KgCO2_perMmbtu  Emissions_stCO2
Plant State Reported\nPrime Mover                                 
NY          CA                       8.513378e+06      7723.206022
            CT                       1.282807e+08    116374.278684

In [40]:
rggi_or_pjm = rggi_or_pjm.groupby(['Plant Id','Plant Name','Reported\nPrime Mover','Plant State', 'Balancing\nAuthority Code','RGGI_state','YEAR'])[gen_cols].sum().stack().to_frame('Generation MWh')

In [42]:
rggi_or_pjm

Generation MWh
Plant Id Plant Name                 Reported\nPrime Mover Plant State Balancing\nAuthority Code  RGGI_state YEAR                                  
54       J K Smith                  GT                    KY          PJM                        0          2019 Netgen\nJanuary         53058.000
                                                                                                                 Netgen\nFebruary        22469.000
                                                                                                                 Netgen\nMarch           21959.000
                                                                                                                 Netgen\nApril           10545.000
                                                                                                                 Netgen\nMay              4981.000
...                                                                                                                                            ...
99999    State-Fuel Level Increment WT                    RI          State-Fuel Level Increment 1          2024 Netgen\nAugust           2886.634
                                                                                                                 Netgen\nSeptember        3108.214
                                                                                                                 Netgen\nOctober             0.000
                                                                                                                 Netgen\nNovember            0.000
                                                                                                                 Netgen\nDecember            0.000

[231396 rows x 1 columns]

In [43]:
rggi_or_pjm['month'] = [x[-1] for x in rggi_or_pjm.reset_index()['level_7'].str.split('\n')]

In [44]:
rggi_or_pjm = rggi_or_pjm.reset_index()

In [45]:
rggi_or_pjm['Date'] = pd.to_datetime('01/'+rggi_or_pjm['month']+'/'+rggi_or_pjm['YEAR'].astype(str),format='%d/%B/%Y')+MonthEnd(0)

In [46]:
rggi_or_pjm#['Reported\nPrime Mover'].unique()

,Plant Id,Plant Name,Reported\nPrime Mover,Plant State,Balancing\nAuthority Code,RGGI_state,YEAR,level_7,Generation MWh,month,Date
0,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nJanuary,53058.000,January,2019-01-31
1,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nFebruary,22469.000,February,2019-02-28
2,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nMarch,21959.000,March,2019-03-31
3,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nApril,10545.000,April,2019-04-30
4,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nMay,4981.000,May,2019-05-31
...,...,...,...,...,...,...,...,...,...,...,...
231391,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nAugust,2886.634,August,2024-08-31
231392,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nSeptember,3108.214,September,2024-09-30
231393,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nOctober,0.000,October,2024-10-31
231394,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nNovember,0.000,November,2024-11-30


In [47]:
rggi_or_pjm[rggi_or_pjm['Reported\nPrime Mover'].isin(["HY",'HK','HA','HB'])].query('RGGI_state==1').groupby('Date')['Generation MWh'].sum().resample('Y').sum().iloc[:-1].describe().to_frame()

,Generation MWh
count,5.000000e+00
mean,3.807308e+07
std,1.781857e+06
min,3.581851e+07
25%,3.715907e+07
50%,3.795136e+07
75%,3.889553e+07
max,4.054093e+07


In [48]:
RGGI_annual_hydro = rggi_or_pjm[rggi_or_pjm['Reported\nPrime Mover'].isin(["HY",'HK','HA','HB'])].query('RGGI_state==1').groupby('Date')['Generation MWh'].sum().resample('Y').sum().iloc[:-1]

In [49]:
all_rggi_gen = rggi_or_pjm.query('RGGI_state==1').groupby('Date')['Generation MWh'].sum().resample('Y').sum().iloc[:-1]
all_rggi_gen.index = all_rggi_gen.index.year

In [50]:
RGGI_annual_hydro.index = RGGI_annual_hydro.index.year

In [51]:
RGGI_annual_hydro/all_rggi_gen

Date
2019    0.116906
2020    0.115501
2021    0.111980
2022    0.105876
2023    0.117278
Name: Generation MWh, dtype: float64

In [52]:
fig = px.bar((RGGI_annual_hydro/all_rggi_gen),labels={'value':'Generation (% of total)'},text_auto=True,title='RGGI State Hydro Generation',width=900)
fig.update_layout(yaxis_tickformat='0.1%')

In [53]:
rggi_or_pjm['Date'] = pd.to_datetime('01/'+rggi_or_pjm['month']+'/'+rggi_or_pjm['YEAR'].astype(str))+MonthEnd(0)

In [54]:
time_series_historical_capacity

Nameplate Capacity (MW)
Plant ID Energy Source Code Prime Mover Code Plant State report_month report_year                         
1.0      DFO                IC               AK          9            2024                             2.7
         WND                WT               AK          9            2024                             1.0
2.0      WAT                HY               AL          9            2024                            53.9
3.0      BIT                ST               AL          9            2024                          1192.5
         NG                 CA               AL          9            2024                           700.4
...                                                                                                    ...
62264.0  SUN                PV               MA          11           2018                             1.2
62265.0  SUN                PV               MA          11           2018                             1.9
62266.0  SUN                PV               MA          11           2018                             1.1
62270.0  DFO                IC               NY          11           2018                             8.0
62271.0  DFO                IC               NY          11           2018                             4.5

[879099 rows x 1 columns]

In [55]:
rggi_or_pjm['month'] = rggi_or_pjm['Date'].dt.month

In [56]:
time_series_historical_capacity_byprime_mover = time_series_historical_capacity.reset_index().groupby(['Plant ID', 'Prime Mover Code','Plant State','report_month','report_year']).sum()

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_19200\1199171893.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [57]:
rggi_or_pjm_capacity = rggi_or_pjm.merge(time_series_historical_capacity_byprime_mover,left_on=['Plant Id','Reported\nPrime Mover','Plant State','month','YEAR'],right_index=True,how='left')

In [58]:
gap_fill = rggi_or_pjm_capacity.groupby(['Plant Id','Reported\nPrime Mover','YEAR'])['Nameplate Capacity (MW)'].mean().to_frame('Gap_fill_capacity_year')

In [59]:
rggi_or_pjm_capacity = rggi_or_pjm_capacity.merge(gap_fill,left_on=['Plant Id','Reported\nPrime Mover','YEAR'],right_index=True,how='left')

In [60]:
rggi_or_pjm_capacity['Nameplate Capacity (MW)'] = np.where((rggi_or_pjm_capacity['Generation MWh']>0) & (pd.isna(rggi_or_pjm_capacity['Nameplate Capacity (MW)'])),
                                                           rggi_or_pjm_capacity['Gap_fill_capacity_year'],rggi_or_pjm_capacity['Nameplate Capacity (MW)'])


In [61]:
gap_fill = rggi_or_pjm_capacity.groupby(['Plant Id','Reported\nPrime Mover'])['Nameplate Capacity (MW)'].mean().to_frame('Gap_fill_capacity_overall')

In [62]:
rggi_or_pjm_capacity = rggi_or_pjm_capacity.merge(gap_fill,left_on=['Plant Id','Reported\nPrime Mover'],right_index=True,how='left')

rggi_or_pjm_capacity['Nameplate Capacity (MW)'] = np.where((rggi_or_pjm_capacity['Generation MWh']>0) & (pd.isna(rggi_or_pjm_capacity['Nameplate Capacity (MW)'])),
                                                           rggi_or_pjm_capacity['Gap_fill_capacity_overall'],rggi_or_pjm_capacity['Nameplate Capacity (MW)'])


In [63]:
rggi_or_pjm_capacity = rggi_or_pjm_capacity.join(plant_emissions,on=['Plant Id','Plant State','Reported\nPrime Mover','Date'])

In [64]:
rggi_or_pjm_capacity[rggi_or_pjm_capacity['Plant Id']==55375].query('YEAR==2020')

,Plant Id,Plant Name,Reported\nPrime Mover,Plant State,Balancing\nAuthority Code,RGGI_state,YEAR,level_7,Generation MWh,month,Date,Nameplate Capacity (MW),Gap_fill_capacity_year,Gap_fill_capacity_overall,KgCO2_perMmbtu,Emissions_stCO2
78672,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nJanuary,100228.998,1,2020-01-31,255.0,255.0,255.0,3.271954e+05,296.826697
78673,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nFebruary,89244.000,2,2020-02-29,255.0,255.0,255.0,3.328039e+04,30.191461
78674,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nMarch,83470.000,3,2020-03-31,255.0,255.0,255.0,3.487298e+05,316.362348
78675,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nApril,78060.000,4,2020-04-30,255.0,255.0,255.0,5.253434e+05,476.583486
78676,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nMay,111394.000,5,2020-05-31,255.0,255.0,255.0,6.453962e+05,585.493540
78677,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nJune,117260.000,6,2020-06-30,255.0,255.0,255.0,2.310051e+06,2095.642561
78678,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nJuly,129515.000,7,2020-07-31,255.0,255.0,255.0,5.009201e+06,4544.270815
78679,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nAugust,133404.000,8,2020-08-31,255.0,255.0,255.0,5.898566e+06,5351.088401
78680,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nSeptember,119460.003,9,2020-09-30,255.0,255.0,255.0,2.717246e+06,2465.043961
78681,55375,Astoria Energy,CA,NY,NYIS,1,2020,Netgen\nOctober,6323.000,10,2020-10-31,255.0,255.0,255.0,4.740736e+04,43.007232


In [65]:
#rggi_or_pjm_capacity[rggi_or_pjm_capacity['Balancing\nAuthority Code']=="PJM"]

In [66]:
px.bar(rggi_or_pjm_capacity.query('`Nameplate Capacity (MW)`>25 ').groupby(['YEAR','RGGI_state'])['Emissions_stCO2'].sum().unstack('RGGI_state'),barmode='group')

In [67]:
(rggi_or_pjm_capacity.query('`Nameplate Capacity (MW)`>25 and RGGI_state==1').groupby(['Plant State','YEAR','RGGI_state'])['Emissions_stCO2'].sum().unstack('YEAR')/10**6).sort_values('RGGI_state').sum()

YEAR
2019    69.077059
2020    65.118495
2021    68.884742
2022    72.339056
2023    67.851164
2024    47.049698
dtype: float64

In [68]:
px.bar(rggi_or_pjm_capacity.query('`Plant Name`=="Brandon Shores"').pivot_table(index='YEAR',columns='month',values='Emissions_stCO2').sum(axis=1))

In [69]:
rggi_or_pjm_capacity

,Plant Id,Plant Name,Reported\nPrime Mover,Plant State,Balancing\nAuthority Code,RGGI_state,YEAR,level_7,Generation MWh,month,Date,Nameplate Capacity (MW),Gap_fill_capacity_year,Gap_fill_capacity_overall,KgCO2_perMmbtu,Emissions_stCO2
0,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nJanuary,53058.000,1,2019-01-31,1055.0,1055.0,1055.0,41687101.17,37817.900369
1,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nFebruary,22469.000,2,2019-02-28,1055.0,1055.0,1055.0,11567448.75,10493.812524
2,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nMarch,21959.000,3,2019-03-31,1055.0,1055.0,1055.0,15713795.90,14255.315219
3,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nApril,10545.000,4,2019-04-30,1055.0,1055.0,1055.0,6997734.59,6348.237755
4,54,J K Smith,GT,KY,PJM,0,2019,Netgen\nMay,4981.000,5,2019-05-31,1055.0,1055.0,1055.0,3051944.61,2768.677455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231391,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nAugust,2886.634,8,2024-08-31,NaN,NaN,NaN,0.00,0.000000
231392,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nSeptember,3108.214,9,2024-09-30,NaN,NaN,NaN,0.00,0.000000
231393,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nOctober,0.000,10,2024-10-31,NaN,NaN,NaN,0.00,0.000000
231394,99999,State-Fuel Level Increment,WT,RI,State-Fuel Level Increment,1,2024,Netgen\nNovember,0.000,11,2024-11-30,NaN,NaN,NaN,0.00,0.000000


In [70]:
px.line(rggi_or_pjm_capacity.reset_index().query('`Plant Name`=="Brandon Shores"'),x='Date',y='Emissions_stCO2')

## RGGI Compliance Data Compare

In [71]:
import os

In [75]:
RGGI_compliance = pd.read_excel(fr'{os.environ["USERPROFILE"]}\OneDrive - Carbon Cap\Carbon markets research\US Markets\RGGI\RGGI Model.xlsx',sheet_name='R2. RGGI Emissions',header=5)

In [76]:
RGGI_compliance = RGGI_compliance[[ 'Pland Generator ID',
                                                                                        'Year',
                                                                                         'Qtr',
                                                                                 'Source Name',
                                                                                   'ORIS Code',
                                                                                  'Unit ID(s)',
                                                                                       'State',
                                                                                  'State Name',
                                                                            'State Identifier',
                                                                              'Control Period',
                                                                                     'Op Time',
                                                                                    'Op Hours',
                                                                             'CO2 Mass (Tons)',
                                                                          'Heat Input (mmBtu)',
                                                       'Eligible Biomass (Tons) (State Value)',
                                            'Eligible CHP Thermal Output (Tons) (State Value)',
                                                                                           'N',
                                                                 'Compliance Emissions (Tons)',
                                                            'CO2 efficiency (tons/heat input)',
                                                                        'IN EIA 2018 database',
                                                                                  'Technology',
                                                                        'Summer Capacity (MW)',
                           'not subject to 2009-2012 RGGI CO2 allowance compliance obligation',
                           'not subject to 2009-2020 RGGI CO2 allowance compliance obligation',
                           'not subject to 2018-2020 RGGI CO2 allowance compliance obligation',
                           'not subject to 2018-2022 RGGI CO2 allowance compliance obligation',
       '(MD LIESA source - not subject to 2018-2023 RGGI CO2 allowance compliance obligation)',
                                                                       'Quarter Auction Price']]

In [77]:
RGGI_compliance.head()

,Pland Generator ID,Year,Qtr,Source Name,ORIS Code,Unit ID(s),State,State Name,State Identifier,Control Period,...,CO2 efficiency (tons/heat input),IN EIA 2018 database,Technology,Summer Capacity (MW),not subject to 2009-2012 RGGI CO2 allowance compliance obligation,not subject to 2009-2020 RGGI CO2 allowance compliance obligation,not subject to 2018-2020 RGGI CO2 allowance compliance obligation,not subject to 2018-2022 RGGI CO2 allowance compliance obligation,(MD LIESA source - not subject to 2018-2023 RGGI CO2 allowance compliance obligation),Quarter Auction Price
0,544_11,2009,1,Devon,544,11,CT,Connecticut,105-14,01/01/2009 - 12/31/2011,...,0.081132,1,Petroleum Liquids,29.2,False,False,False,False,False,3.51
1,544_11,2024,3,Devon,544,11,CT,Connecticut,105-14,01/01/2024 - 12/31/2026,...,0.081011,1,Petroleum Liquids,29.2,False,False,False,False,False,25.75
2,544_12,2024,3,Devon,544,12,CT,Connecticut,105-14,01/01/2024 - 12/31/2026,...,0.081015,1,Petroleum Liquids,29.2,False,False,False,False,False,25.75
3,544_13,2024,3,Devon,544,13,CT,Connecticut,105-14,01/01/2024 - 12/31/2026,...,0.081008,1,Petroleum Liquids,29.9,False,False,False,False,False,25.75
4,544_14,2024,3,Devon,544,14,CT,Connecticut,105-14,01/01/2024 - 12/31/2026,...,0.081006,1,Petroleum Liquids,29.7,False,False,False,False,False,25.75


In [78]:
RGGI_compliance.query('State!="VA"').groupby(['ORIS Code','Year'])['Compliance Emissions (Tons)'].sum().unstack('Year').iloc[:,-4:-1].fillna(0).sort_values(2023,ascending=False).head(10)

Year,2021,2022,2023
ORIS Code,,,
60357,0.000,1817853.685,4084069.753
55524,0.000,1825494.528,3897636.392
55375,2572648.672,3207104.264,3453477.303
55298,0.000,1728280.508,3292674.387
60368,0.000,1711278.738,3199586.202
50006,1663089.926,2983940.236,3096556.054
3118,0.000,2301422.053,3069606.847
3136,0.000,1708322.542,3039819.772
55690,0.000,1147722.616,2904928.771


In [79]:
rggi_or_pjm_capacity['Qtr'] = rggi_or_pjm_capacity.Date.dt.quarter

In [80]:
test = RGGI_compliance.query('`ORIS Code`==60357 and Year>2019').sort_values(['Year','Qtr']).groupby(['Year','Qtr'])['Compliance Emissions (Tons)'].sum()


In [81]:
test

Year  Qtr
2022  3      1069504.843
      4       748348.842
2023  1      1175741.265
      2       802061.311
      3      1156640.016
      4       949627.161
Name: Compliance Emissions (Tons), dtype: float64

In [82]:
rggi_or_pjm_capacity.query('`Plant Id`==55375 and YEAR==2021 and Qtr==1')

,Plant Id,Plant Name,Reported\nPrime Mover,Plant State,Balancing\nAuthority Code,RGGI_state,YEAR,level_7,Generation MWh,month,Date,Nameplate Capacity (MW),Gap_fill_capacity_year,Gap_fill_capacity_overall,KgCO2_perMmbtu,Emissions_stCO2,Qtr
78684,55375,Astoria Energy,CA,NY,NYIS,1,2021,Netgen\nJanuary,0.000,1,2021-01-31,255.0,255.0,255.0,0.00,0.000000,1
78685,55375,Astoria Energy,CA,NY,NYIS,1,2021,Netgen\nFebruary,0.000,2,2021-02-28,255.0,255.0,255.0,0.00,0.000000,1
78686,55375,Astoria Energy,CA,NY,NYIS,1,2021,Netgen\nMarch,78214.000,3,2021-03-31,255.0,255.0,255.0,886824.51,804.513627,1
78756,55375,Astoria Energy,CT,NY,NYIS,1,2021,Netgen\nJanuary,0.000,1,2021-01-31,340.0,340.0,340.0,0.00,0.000000,1
78757,55375,Astoria Energy,CT,NY,NYIS,1,2021,Netgen\nFebruary,-947.000,2,2021-02-28,340.0,340.0,340.0,172880.95,156.834953,1
78758,55375,Astoria Energy,CT,NY,NYIS,1,2021,Netgen\nMarch,133482.005,3,2021-03-31,340.0,340.0,340.0,82162890.04,74536.916752,1


In [83]:
compare = rggi_or_pjm_capacity.query('`Plant Id`==60357 and YEAR>2019').set_index('Date').groupby(['YEAR','Qtr'])[['Generation MWh','Emissions_stCO2']].sum().reset_index().join(test,on=['YEAR','Qtr'])

In [84]:
compare

,YEAR,Qtr,Generation MWh,Emissions_stCO2,Compliance Emissions (Tons)
0,2020,1,2389041.0,770254.239114,NaN
1,2020,2,1785554.0,572313.887394,NaN
2,2020,3,2608856.0,832990.558306,NaN
3,2020,4,2442891.0,764962.381655,NaN
4,2021,1,3069031.0,950114.945834,NaN
5,2021,2,2026436.0,630396.017735,NaN
6,2021,3,2793979.0,873085.202000,NaN
7,2021,4,2665876.0,828493.422649,NaN
8,2022,1,2848297.0,891287.580807,NaN
9,2022,2,1692924.0,579010.775752,NaN


In [85]:
px.scatter(compare,x='Emissions_stCO2',y='Compliance Emissions (Tons)',hover_data=['YEAR','Qtr'],trendline='ols')

In [86]:
px.bar(compare[['Emissions_stCO2','Compliance Emissions (Tons)']],barmode='group')

### PJM Compare

In [87]:
pjm = rggi_or_pjm_capacity[rggi_or_pjm_capacity['Balancing\nAuthority Code']=='PJM']

In [88]:
pjm['In_RGGI'] = np.where((pjm['RGGI_state']==1) ,1,0 ) #& (pjm['Nameplate Capacity (MW)']>=25)

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_19200\2863188609.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
px.bar(pjm.groupby(['Date','In_RGGI'])['Emissions_stCO2'].sum().unstack())

In [ ]:
pjm.columns

In [ ]:
px.bar(pjm.query('Emissions_stCO2>0').groupby(['Date','In_RGGI'])['Nameplate Capacity (MW)'].count().unstack())

In [ ]:
list(pjm.query('YEAR==2023 and Emissions_stCO2>0')['Plant Id'][~(pjm.query('YEAR==2023 and Emissions_stCO2>0')['Plant Id'].isin(pjm.query('YEAR==2024 and Emissions_stCO2>0')['Plant Id'].unique()))].unique())

In [ ]:
pjm.query('YEAR==2023 and `Plant Id`==1353')

In [ ]:
pjm.query('YEAR==2024 and `Plant Id`==1353')

In [ ]:
mega_gen_fuel_df.query('RGGI_state==1 or `Balancing\nAuthority Code`=="PJM" ')

In [ ]:
mega_gen_fuel_df[gen_cols] = mega_gen_fuel_df[gen_cols].replace('...',0).replace('.',0)

In [ ]:
PJM_compare = mega_gen_fuel_df[mega_gen_fuel_df['Balancing\nAuthority Code']=='PJM'].groupby(['RGGI_state','YEAR','Reported\nFuel Type Code'])[gen_cols].mean()

In [ ]:
PJM_compare.columns = [x.split('\n')[-1] for x in PJM_compare.columns]

In [ ]:
PJM_compare = PJM_compare.stack().to_frame('Generation').reset_index()

In [ ]:
PJM_compare['full_date'] = '01/'+PJM_compare['level_3']+'/'+PJM_compare['YEAR'].astype(str) 

In [ ]:
PJM_compare['date'] = pd.to_datetime(PJM_compare['full_date'],format='%d/%B/%Y')

In [ ]:
PJM_compare_pivot = PJM_compare.pivot_table(index='date',columns=['Reported\nFuel Type Code','RGGI_state'],values='Generation')

In [ ]:
PJM_compare_pivot.columns.get_level_values(0).unique()

In [ ]:
px.line(PJM_compare_pivot['BIT'].cumsum())

In [ ]:
gen_cols.append('YEAR')

In [ ]:
px.line(mega_gen_fuel_df.query('`Plant Name`=="Brandon Shores"')[gen_cols].groupby('YEAR').sum().transpose().replace('...',0).cumsum())